In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt
import re
import io
import urllib.request
from konlpy.tag import Okt
from konlpy.tag import Mecab
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from keras_preprocessing.text import tokenizer_from_json

# 1.Load model

In [2]:
model = keras.models.load_model('./gru_review_saved_model_0621')

2022-06-24 04:10:39.415290: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-24 04:10:39.527768: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-24 04:10:39.528432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-24 04:10:39.530918: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [3]:
okt = Okt()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '좀', '으로', '하다', '걍', '네요', '잘']

maxlen = 80

with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [4]:
import bentoml
from bentoml import BentoService, api, artifacts, env
from bentoml.frameworks.keras import KerasModelArtifact
from bentoml.frameworks.xgboost import XgboostModelArtifact
from bentoml.frameworks.pytorch import PytorchModelArtifact
from bentoml.adapters import JsonInput, JsonOutput

In [18]:
@env(infer_pip_packages=True)
@artifacts([KerasModelArtifact('model')])

class KerasModelService(bentoml.BentoService):
    @api(input=ModelApiInputValidator())
    
    def predict(self, new_sentence):
        
        new_sentence = re.sub(r'<.+?>', '', new_sentence)
        new_sentence = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z]', ' ', new_sentence)
        new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
        new_sentence = re.sub('\s+', ' ',new_sentence)

        new_sentence = okt.morphs(new_sentence)  
        new_sentence = [word for word in new_sentence if not word in stopwords]

        encoded = tokenizer.texts_to_sequences([new_sentence])

        pad_new = pad_sequences(encoded, maxlen=maxlen)

        score = float(gru_loaded_model.predict(pad_new))

        return score

NameError: name 'ModelApiInputValidator' is not defined

In [ ]:
svc = KerasModelService()
svc.pack('model', model_to_save)